In [2]:
import os

os.chdir('Z:\OMOP\omop_etl')
print(os.getcwd())

import pandas as pd
from omop_etl.datastore import DataStore
import pandas as pd
from turbodbc import connect, make_options, Megabytes, Rows
import time
import math
import numpy as np
from pydma.databases import OneFLDb
from pydma.utils import find

Z:\OMOP\omop_etl


In [3]:
#from https://stackoverflow.com/questions/5478351/python-time-measure-function
from contextlib import contextmanager

@contextmanager
def timeit_context(name=None):
    startTime = time.time()
    yield
    elapsedTime = time.time() - startTime
    print('{} finished in {}'.format(name, 
                                     time.strftime("%H h %M m %S s", 
                                     time.gmtime(elapsedTime))))

In [24]:
# %%timeit
# table = "measurement"
# batch_size = 1_000_000
# omop = OneFLDb('edw', database='DWS_OMOP')
# tables = ['measurement']#'device_exposure', 'person', 'death', 'location', 'provider', 'care_site', 'visit_occurrence', 
# #           'drug_exposure','condition_occurrence', 'observation', 'procedure_occurrence', 'observation_period']
# rpath = '//share.***REMOVED***/COVID-19/OMOP/data_releases/v7'
# #omop.list_tables(in_schema=['hipaa'])
# # print(tables.Table.to_list())
# # n_batches = 40

# def nrows(table):
#     """Returns table number of rows.
#     """
#     con = connect(driver='{SQL Server}', server='edw.***REMOVED***.edu', 
#                   database='DWS_OMOP', trusted_connection='yes')
#     cursor= con.cursor()
#     con.autocommit = True
#     cursor.execute("EXEC sp_spaceused N'hipaa.{}';".format(table))
#     result = cursor.fetchall()[0]
#     con.close()
    
#     return result

# table = 'drug_exposure'
# # for table in tables: 
# results = nrows(table)
# kb_row = int(results[2].split(' ')[0])/int(results[1])
# # n_batches = math.ceil(rows/batch_size)
# # # batch_size = math.ceil(rows/n_batches)
# # print('Total rows: ', rows, '\nBatch size: ', batch_size, '\nBatches:', n_batches)
# kb_row*1e6/1000

122.54599732155617

In [ ]:
# table = 'visit_occurrence'
rpath = '//share.***REMOVED***/COVID-19/OMOP/data_releases/v7'
tables = ['drug_exposure', 'visit_occurrence', 'measurement']

for table in tables:
    # Set batch size in number of rows
    options = make_options(read_buffer_size=Megabytes(2000), 
                           prefer_unicode=True,
                           use_async_io=True,
                           limit_varchar_results_to_max=True)

    con = connect(driver='{SQL Server}', server='edw.***REMOVED***.edu', database='DWS_OMOP', 
                  trusted_connection='yes', turbodbc_options=options)

    cursor = con.cursor()
    cursor.execute(f"select * from hipaa.{table}")
    batches = cursor.fetchnumpybatches()

    count = 0
    file = os.path.join(rpath, table + '.csv')

    with timeit_context(f'Exporting {table}'):
        if os.path.exists(file):
            os.remove(file)

        for batch in batches:

            df = pd.DataFrame(batch)

            if count == 0:
                dtypes = {t:batch[t].dtype.type for t in batch.keys()}
                for dtype in dtypes.keys():
                    if dtypes[dtype] == np.int64:
                        dtypes[dtype] = 'Int64'
                    else:
                        dtypes[dtype] = 'str'

                df = df.astype(dtypes)
                df.to_csv(file, index=False, sep='\t')

            else: 
                df = df.astype(dtypes)
                df.to_csv(file, header=False, index=False, mode='a', sep='\t')

            count=+1

        con.close()

In [20]:
# count = 0
# for batch in batches:
#     if count == 0:
#         df = pd.DataFrame(batch)
#         print('Rows:',df.shape)
#         print('Memory usage (MB):', df.memory_usage(deep=True).sum()/1e6)
#     else: break
#     count+=1

Rows: (7956, 23)
Memory usage (MB): 5.603933


In [46]:
# file = os.path.join(rpath, table + '.csv')
# count = 1
# with timeit_context(f'Exporting {table}'):
#     if os.path.exists(file):
#         os.remove(file)
    
#     with open(file, 'a') as f:
#         with omop.engine.connect() as con:
#             header = pd.read_sql('select top 0 * from {}'.format(table), con)
#             header.to_csv(f, index=False, sep='\t')
#             for chunk in pd.read_sql('select * from {}'.format(table), con, coerce_float=False, chunksize=batch_size):
#                 if count > 1: break
# #                 chunk.to_csv(f, header=False, index=False, sep='\t')
#                 print(f'Batch {count}/{n_batches} complete.', end='\r')
#                 count=+1

In [46]:
# Set batch size in number of rows

# with ncov.engine.connect() as con:
#     drug = pd.read_sql('select * from {}'.format(table), con, chunksize=batch_size)